# Organize your dataset according to applicable standards

In [1]:
from IPython.display import Markdown, display
import ipywidgets as widgets # load library for interactive widgets (dropdown lists, button, etc.)
from ipywidgets import HBox, Label, Layout
from IPython.display import clear_output, HTML # clear output if you click on button several times
import os # to manipulate folders and files
import pandas as pd # for conveniently importing excel files

### Define some useful functions

# Print with style (bold etc.)
def printmd(string): 
    display(Markdown(string))

# Dropdown list
def mydropdownlist(listoptions):
    mydropdownlistis = widgets.Dropdown(
    options = listoptions,
    value = listoptions[0],
    disabled = False    
    )
    return mydropdownlistis

# Button
def mybutton(mydescription):
    mybuttonis = widgets.Button(
    description = mydescription,
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''    
    )
    return mybuttonis

### Data type selection interface ###

# Import name of the checklists currently included in DOPE directly from the "Checklists" folder
datatype_options = ['Select']
current_folder = os.getcwd() 
main_folder = os.path.dirname(current_folder) 
checklists_folder = r'Checklists' 
checklists_path = os.path.join(main_folder, checklists_folder) # full path = main folder/checklists
checklist_names = [".".join(f.split(".")[:-1]) for f in os.listdir(checklists_path) \
                  if os.path.isfile(os.path.join(checklists_path, f))] # Import checklist names without extension
checklist_names = [x for x in checklist_names if not x.startswith('~')] # discard temp excel files starting with ~
datatype_options = datatype_options + checklist_names + ['Other'] # dropdown list options

# Dropdown list for selecting datatype
select_datatype = mydropdownlist(datatype_options)

# Button to validate dataytype selection and display checklist
validate_datatype = mybutton('Validate selection')

# Widget box for nice display
selection_box = HBox([Label('Select your data type:'), select_datatype, validate_datatype])
display(selection_box)

print()
print()
### Display Checklist ###

templates_folder = 'https://github.com/bvhpatel/DOPE/raw/master/templates' #location of the form templates

# Action when button clicked: show checklist as a table
output = widgets.Output()
@output.capture()
def on_button_clicked(b):
    clear_output()
    if (select_datatype.value == datatype_options[0]): # if "Select" is selected
        printmd('**Please select data type**')
    elif(select_datatype.value == "Other"): # if "Other" is selected
        printmd('**Your data type has not been doped yet**') 
    else:
        printmd("**Checklist for {}**".format(select_datatype.value))
        selected_checklist_name = select_datatype.value
        selected_checklist_excel = os.path.join(checklists_path, selected_checklist_name + '.xlsx')
        df = pd.read_excel(selected_checklist_excel)
        pd.set_option('display.max_colwidth', -1)
        count = -1
        for web in df['Webpage']:
            count += 1
            if (web != 'N / A'): # link to webpage
                df['Webpage'].iloc[count] = '<a href= {0}>{1}</a>'.format(web, web)
            form = df['Template'].iloc[count]
            if (form != 'N / A'): # link to form template
                formlink = os.path.join(templates_folder, form)
                df['Template'].iloc[count] = '<a href= {0}>{1}</a>'.format(formlink, form)
            interactive_help = df['Interactive help'].iloc[count]
            if (interactive_help != 'N / A'): # link to form template
                df['Interactive help'].iloc[count] = '<a href= {0}>{1}</a>'.format(interactive_help, interactive_help)
        display(HTML(df.to_html(escape=False)))
validate_datatype.on_click(on_button_clicked)
display(output)


Output()